In [1]:
import sys
import pandas as pd
import numpy as np
from scipy.spatial.distance import canberra

sys.path.append("..")

In [2]:
train = pd.read_csv('../data/500_ngrams/train_features_scaled_minmax.csv', index_col=0)
val = pd.read_csv('../data/500_ngrams/validation_features_scaled_minmax.csv', index_col=0)
imposters = pd.read_csv('../data/imposters/imposters_scaled_minmax.csv', index_col=0)

### Use set(top 150 features from selector + top 150 from random forest)

In [4]:
TOP_FEATURES = [
    0, 1, 2, 3, 5, 6, 7, 8, 11, 15, 16, 26, 35, 42, 45, 46, 47, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59,
    60, 63, 66, 68, 69, 81, 84, 88, 91, 95, 97, 98, 99, 103, 108, 109, 110, 111, 112, 114, 119, 120, 121,
    122, 123, 128, 130, 133, 136, 138, 139, 140, 146, 149, 152, 154, 160, 163, 166, 168, 170, 173, 174, 175,
    176, 186, 187, 189, 193, 197, 198, 199, 200, 201, 202, 203, 204, 212, 213, 219, 225, 227, 232, 233, 234,
    236, 240, 241, 242, 243, 244, 248, 251, 256, 263, 273, 276, 279, 282, 283, 288, 290, 292, 294, 299, 310,
    317, 325, 328, 330, 331, 333, 337, 341, 342, 344, 349, 350, 352, 354, 357, 362, 364, 366, 368, 379, 402,
    403, 409, 410, 416, 417, 425, 426, 428, 439, 440, 446, 451, 455, 456, 457, 458, 471, 473, 476, 482, 492,
    496, 499, 501, 502, 503, 507, 508, 512, 514, 515, 516, 521, 524, 531, 537, 543, 544, 552, 561, 562, 563,
    568, 571, 572, 589, 592, 593, 594, 603, 608, 613, 614, 627, 631, 635, 639, 641, 644, 654, 658, 659, 663,
    665, 672, 673, 674, 675, 676, 683, 684, 690, 694, 699, 704, 707, 710, 712, 722, 725, 726, 729, 732, 734,
    750, 752, 753, 754, 758, 766, 772, 773, 775, 783, 784, 789, 793, 797, 808, 812, 813, 826, 828, 835, 837,
    839, 840, 846, 849, 850, 866, 870, 882, 884, 894, 898, 899, 902, 908, 909, 914, 915, 918, 919, 925
]

a_columns = ['A_{}'.format(i) for i in TOP_FEATURES]
b_columns = ['B_{}'.format(i) for i in TOP_FEATURES]
i_columns = ['I_{}'.format(i) for i in TOP_FEATURES]

train_A = train[a_columns]
train_B = train[b_columns]
val_A = val[a_columns]
val_B = val[b_columns]
imposters = imposters[i_columns]

In [5]:
NUMBER_OF_IMPOSTERS = 5000

imposters_indices = ['I_{}'.format(i) for i in range(NUMBER_OF_IMPOSTERS)]
train_A_indices = ['A_{}'.format(i) for i in train.index]
train_B_indices = ['B_{}'.format(i) for i in train.index]

imposters_similarities = pd.DataFrame(columns=imposters_indices,
                                      index=train_A_indices+train_B_indices)

In [6]:
check = 0

for imposter_index in range(NUMBER_OF_IMPOSTERS):
    imposter = imposters.loc[imposter_index]
    for train_index in train.index:
        A = train_A.loc[train_index]
        B = train_B.loc[train_index]
        imposters_similarities.loc['A_{}'.format(train_index)]['I_{}'.format(imposter_index)] = canberra(A, imposter)
        imposters_similarities.loc['B_{}'.format(train_index)]['I_{}'.format(imposter_index)] = canberra(B, imposter)
        
    check += 1
    if check % 100 == 0:
        print(check)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [7]:
imposters_similarities.to_csv('../data/imposters/train_imposters_canberra_similarities_259.csv')

In [19]:
imposters_similarities = pd.read_csv('../data/imposters/train_imposters_canberra_similarities_259.csv', index_col=0)

In [20]:
imposters_similarities.head()

,I_0,I_1,I_2,I_3,I_4,I_5,I_6,I_7,I_8,I_9,...,I_4990,I_4991,I_4992,I_4993,I_4994,I_4995,I_4996,I_4997,I_4998,I_4999
A_0,105.220659,83.742488,87.998556,99.856123,105.372616,80.519159,97.476783,95.435466,94.174481,84.375315,...,87.271263,93.368737,99.179971,85.121647,88.698559,88.933205,143.390739,100.505930,90.252838,89.608601
A_1,96.004457,111.278925,101.223115,96.336159,110.284212,105.730793,115.276571,110.611425,111.735268,106.713187,...,121.945558,102.147836,100.199526,126.351208,112.951613,96.437061,115.176816,123.822484,97.714609,114.328905
A_2,105.114791,83.711946,88.812559,100.587283,105.271681,80.592242,97.717408,95.610948,94.177735,85.005759,...,86.559792,93.151652,99.973161,85.182846,88.418783,89.428242,143.148785,100.040700,90.804074,89.684621
A_3,109.066159,83.633282,99.884828,96.984614,106.704516,83.398635,89.680169,86.325686,85.484433,99.322462,...,88.084039,79.139258,97.963188,89.700851,104.794584,96.211721,154.159364,89.574244,87.382637,80.968284
A_4,98.853824,110.108021,102.962678,108.250080,122.240849,103.737341,102.375838,107.016993,90.656635,105.305713,...,110.109632,105.581906,106.008559,108.141370,96.820429,110.423466,130.111287,112.638117,92.856652,103.393096


In [11]:
del imposters_similarities

In [12]:
NUMBER_OF_IMPOSTERS = 5000

imposters_indices = ['I_{}'.format(i) for i in range(NUMBER_OF_IMPOSTERS)]
val_A_indices = ['A_{}'.format(i) for i in val.index]
val_B_indices = ['B_{}'.format(i) for i in val.index]

val_imposters_similarities = pd.DataFrame(columns=imposters_indices,
                                          index=val_A_indices+val_B_indices)

check = 0

for imposter_index in range(NUMBER_OF_IMPOSTERS):
    imposter = imposters.loc[imposter_index]
    for val_index in val.index:
        A = val_A.loc[val_index]
        B = val_B.loc[val_index]
        val_imposters_similarities.loc['A_{}'.format(val_index)]['I_{}'.format(imposter_index)] = canberra(A, imposter)
        val_imposters_similarities.loc['B_{}'.format(val_index)]['I_{}'.format(imposter_index)] = canberra(B, imposter)
        
    check += 1
    if check % 100 == 0:
        print(check)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [13]:
val_imposters_similarities.to_csv('../data/imposters/val_imposters_canberra_similarities_259.csv')

In [14]:
val_imposters_similarities

,I_0,I_1,I_2,I_3,I_4,I_5,I_6,I_7,I_8,I_9,...,I_4990,I_4991,I_4992,I_4993,I_4994,I_4995,I_4996,I_4997,I_4998,I_4999
A_0,101.258,85.0521,98.1138,103.769,104.084,86.8985,103.57,90.6561,92.7948,92.2995,...,91.9247,81.5556,86.6227,91.1553,90.4758,94.5946,136.604,92.8938,84.0613,101.265
A_1,95.5525,96.1929,94.6736,105.342,120.605,93.2308,96.6339,97.5734,90.3002,102.513,...,90.8365,94.499,92.6889,97.9223,93.6828,108.35,143.448,94.4193,91.0937,102.435
A_2,89.6861,97.0434,92.2889,87.9579,102.093,89.6075,110.573,99.4032,92.2666,108.04,...,104.543,91.8908,87.6231,106.078,93.7026,96.5488,130.22,110.33,93.153,105.853
A_3,104.203,83.9085,92.6172,92.4637,94.1824,82.6377,82.5598,83.48,83.7172,98.2252,...,82.311,84.6865,94.6546,96.6168,94.2633,84.4964,136.026,79.0718,76.4059,89.8436
A_4,105.976,85.2277,88.7343,101.383,101.763,100.563,107.748,106.851,97.661,96.2845,...,101.513,87.0732,99.9005,98.0559,88.6183,88.1005,134.478,101.007,94.6683,92.1451
A_5,104.879,105.3,96.9263,101.298,100.792,93.8655,108.176,111.683,91.1757,105.34,...,100.105,107.369,102.011,115.191,103.561,104.379,126.093,100.434,90.5532,114.254
A_6,106.404,93.4436,96.2783,99.7511,93.5646,103.082,110.488,105.379,100.375,100.575,...,110.253,88.5914,98.9294,105.686,91.6127,89.3477,129.427,107.42,100.805,95.574
A_7,115.712,90.6459,93.0786,96.8213,106.352,79.8098,101.331,95.2999,100.377,84.8505,...,103.088,83.9943,92.1037,94.269,85.825,85.8437,138.067,100.982,80.2837,97.1108
A_8,115.766,93.6168,97.2421,86.7912,96.5853,92.0427,102.768,100.338,101.794,99.2931,...,101.954,92.6537,100.194,107.083,100.246,89.6116,129.198,104.799,93.2695,100.994
A_9,110.323,92.077,95.934,91.1751,112.206,90.8726,101.582,95.7052,88.6369,97.5678,...,83.2477,95.2955,96.4424,84.5686,91.6969,101.246,142.761,94.371,88.063,97.7747


In [15]:
# choose random 1000 imposters of these 5000 as files are very large

In [24]:
import random

random_1000_imposter_indices = random.sample(imposters_similarities.columns.tolist(), 1000)

In [28]:
subset_imposters_similarities = imposters_similarities[random_1000_imposter_indices]
subset_val_imposters_similarities = val_imposters_similarities[random_1000_imposter_indices]

In [32]:
subset_imposters_similarities.to_csv('../data/imposters/1000_train_imposters_canberra_similarities_259.csv')
subset_val_imposters_similarities.to_csv('../data/imposters/1000_val_imposters_canberra_similarities_259.csv')

In [38]:
indices = [i[2:] for i in random_1000_imposter_indices]

In [42]:
indices = [i[2:] for i in random_1000_imposter_indices]

subset_imposters_features = imposters.iloc[indices]
subset_imposters_features.to_csv('../data/imposters/1000_imposters_features.csv')

In [43]:
pd.read_csv('../data/imposters/1000_imposters_features.csv', index_col=0)

,I_0,I_1,I_2,I_3,I_5,I_6,I_7,I_8,I_11,I_15,...,I_898,I_899,I_902,I_908,I_909,I_914,I_915,I_918,I_919,I_925
1726,0.541695,0.321407,0.394291,0.325552,0.378572,0.000000,0.183234,0.493926,0.173760,0.117987,...,0.000000,0.000000,0.000000,0.000000,0.083080,0.000000,0.000000,0.000000,0.000000,0.000000
2620,0.428808,0.431176,0.590120,0.538529,0.269629,0.000000,0.000000,0.606110,0.140759,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1188,0.459401,0.218414,0.155524,0.567711,0.480418,0.181754,0.000000,0.271647,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.273632
470,0.240263,0.281294,0.619048,0.596313,0.581498,0.000000,0.142730,0.215424,0.244551,0.000000,...,0.000000,0.000000,0.000000,0.362106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4367,0.599659,0.179496,0.310437,0.528821,0.417806,0.068846,0.082178,0.418880,0.205174,0.000000,...,0.000000,0.000000,0.122881,0.000000,0.000000,0.000000,0.092578,0.000000,0.000000,0.258945
598,0.768336,0.305069,0.183752,0.547190,0.617709,0.023222,0.394999,0.625742,0.080779,0.023857,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
792,0.700828,0.381723,0.150519,0.549441,0.691673,0.282033,0.168323,0.440305,0.186235,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1310,0.581446,0.253099,0.313718,0.676689,0.540817,0.000000,0.083442,0.304664,0.132298,0.000000,...,0.000000,0.000000,0.118852,0.000000,0.379533,0.000000,0.000000,0.000000,0.000000,0.000000
195,0.499566,0.216401,0.451943,0.628468,0.478621,0.000000,0.167744,0.489140,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
728,0.660986,0.337929,0.406717,0.391780,0.412426,0.104058,0.393324,0.328072,0.181217,0.147575,...,0.000000,0.282934,0.130240,0.000000,0.207948,0.000000,0.000000,0.000000,0.000000,0.082335
